In [5]:
import os
import torch
from tqdm.auto import tqdm
import warnings
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd

# 忽略特定的警告
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned*")

# 加载新的数据集
new_file_path = './dataset/Eclipse_morethan10_processed_again.csv'
# 指定需要提取的列
columns_to_extract = ['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority', 'developer',  'status']
# columns_to_extract = [ 'description', 'developer']
df = pd.read_csv(new_file_path, usecols=columns_to_extract, encoding='latin-1')

# 将developer列作为标签
label_dict = {label: idx for idx, label in enumerate(df['developer'].unique())}
df['label'] = df['developer'].replace(label_dict)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19256\580160181.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['developer'].replace(label_dict)


In [6]:
# 合并文本信息为模型的输入，除了developer列
# df['text_input'] = df[['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority',  'status']].astype(str).agg(' '.join, axis=1)
df['text_input'] =   df['abstracts'].astype(str)+ " " + df['description'].astype(str)  # 使用空格作为分隔符

# df['text_input'] = df[[ 'product',   'component', 'severity', 'priority',  'status','description']].astype(str).agg(' '.join, axis=1)
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15, random_state=42, stratify=df.label.values)
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [7]:

# 对训练和验证数据的合并文本进行编码
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_input.values,  # 使用合并后的文本
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_input.values,  # 使用合并后的文本
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

# 准备Tensor数据
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# 定义DataLoader
batch_size = 4
train_loader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
val_loader = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=32)

In [8]:
# # 初始化模型
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)
# optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# # 加载模型
# # 计算层数
# num_transformer_layers = len(model.bert.encoder.layer)
# print(f'The BERT model has {num_transformer_layers} transformer layers.')
# print(model)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# 加载模型
# 计算层数
num_transformer_layers = len(model.bert.encoder.layer)
print(f'The BERT model has {num_transformer_layers} transformer layers.')
print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\ProgramData\anaconda3\envs\django5\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


The BERT model has 12 transformer layers.
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [9]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

checkpoint_path = 'model_checkpoint__eclipseprocessed_again_withabstract+description_normal.pth'

# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')



Resuming training from epoch 2


In [10]:
import pymysql
from datetime import datetime

# 数据库连接信息
host = '38.147.173.234'
user = 'root'
password = '123456'
db = 'training_statistics_db'

# 模型名称，根据实际情况手动设置
model_name = 'bert-base-cased'
# 学习率和可选特性，根据实际情况手动设置
learning_rate = 1e-5  # 示例学习率
optional_feature = 'abstract+descrition'  # 示例可选特性
dataset = new_file_path
num_epochs=15

for epoch in range(start_epoch, num_epochs):
    model.train()
    start_time = datetime.now()
    # ...训练过程...
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
        
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
    model.eval()
    correct_topk = {k: 0 for k in range(1, 11)}
    total = 0
    val_progress_bar = tqdm(val_loader, desc="Validating")
    
    for batch in val_progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        total += labels.size(0)
        
        # 计算top1到top10的正确率
        _, predicted_topk = torch.topk(logits, k=10, dim=1)
        labels_expanded = labels.unsqueeze(1)
        for k in range(1, 11):
            correct_topk[k] += (predicted_topk[:, :k] == labels_expanded).any(dim=1).sum().item()
                
    # 打印每个topK的准确率
    top10accuracy = []  # 初始化存储Top1到Top10准确率的数组

    for k in range(1, 11):
        accuracy = 100 * correct_topk[k] / total
        top10accuracy.append(accuracy)  # 将计算出的准确率添加到数组中
        print(f'Accuracy after epoch {epoch + 1}: Top{k}: {accuracy:.2f}%')
        print(top10accuracy)
    import pandas as pd
    import os

    # 定义数据字典，用于创建DataFrame
    data = {
            'epoch': [epoch],
            'start_time': [start_time],
            'end_time': [end_time],
            'duration': [duration],
            'user_id': [1],
            'model': [model_name],
            'top1_accuracy': [top10accuracy[0]],
            'top2_accuracy': [top10accuracy[1]],
            'top3_accuracy': [top10accuracy[2]],
            'top4_accuracy': [top10accuracy[3]],
            'top5_accuracy': [top10accuracy[4]],
            'top6_accuracy': [top10accuracy[5]],
            'top7_accuracy': [top10accuracy[6]],
            'top8_accuracy': [top10accuracy[7]],
            'top9_accuracy': [top10accuracy[8]],
            'top10_accuracy': [top10accuracy[9]],
            'optional_feature': [optional_feature],
            'learning_rate': [learning_rate],
            'dataset': [dataset]
    }

        # 创建DataFrame
    df = pd.DataFrame(data)

        # 检查train.csv文件是否存在来决定是否添加表头
    file_exists = os.path.isfile('train.csv')

        # 如果文件存在，不写入表头，模式为追加；如果文件不存在，写入表头，模式为写入
    df.to_csv('train.csv', mode='a', header=not file_exists, index=False)

    print(f'Epoch {epoch + 1} training data inserted into train.csv.')

Epoch 3:   0%|          | 0/8599 [00:00<?, ?it/s]

Validating:   0%|          | 0/190 [00:00<?, ?it/s]

Accuracy after epoch 3: Top1: 39.49%
[39.489291598023065]
Accuracy after epoch 3: Top2: 53.82%
[39.489291598023065, 53.82207578253707]
Accuracy after epoch 3: Top3: 62.03%
[39.489291598023065, 53.82207578253707, 62.026359143327845]
Accuracy after epoch 3: Top4: 66.80%
[39.489291598023065, 53.82207578253707, 62.026359143327845, 66.80395387149917]
Accuracy after epoch 3: Top5: 71.07%
[39.489291598023065, 53.82207578253707, 62.026359143327845, 66.80395387149917, 71.07084019769357]
Accuracy after epoch 3: Top6: 73.92%
[39.489291598023065, 53.82207578253707, 62.026359143327845, 66.80395387149917, 71.07084019769357, 73.92092257001647]
Accuracy after epoch 3: Top7: 76.33%
[39.489291598023065, 53.82207578253707, 62.026359143327845, 66.80395387149917, 71.07084019769357, 73.92092257001647, 76.32619439868205]
Accuracy after epoch 3: Top8: 78.22%
[39.489291598023065, 53.82207578253707, 62.026359143327845, 66.80395387149917, 71.07084019769357, 73.92092257001647, 76.32619439868205, 78.22075782537067

Epoch 4:   0%|          | 0/8599 [00:00<?, ?it/s]

Validating:   0%|          | 0/190 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/190 [00:00<?, ?it/s]

Accuracy after epoch 14: Top1: 47.10%
[47.1004942339374]
Accuracy after epoch 14: Top2: 59.74%
[47.1004942339374, 59.73640856672158]
Accuracy after epoch 14: Top3: 66.99%
[47.1004942339374, 59.73640856672158, 66.9851729818781]
Accuracy after epoch 14: Top4: 71.86%
[47.1004942339374, 59.73640856672158, 66.9851729818781, 71.86161449752883]
Accuracy after epoch 14: Top5: 74.70%
[47.1004942339374, 59.73640856672158, 66.9851729818781, 71.86161449752883, 74.69522240527183]
Accuracy after epoch 14: Top6: 77.25%
[47.1004942339374, 59.73640856672158, 66.9851729818781, 71.86161449752883, 74.69522240527183, 77.2487644151565]
Accuracy after epoch 14: Top7: 79.24%
[47.1004942339374, 59.73640856672158, 66.9851729818781, 71.86161449752883, 74.69522240527183, 77.2487644151565, 79.24217462932455]
Accuracy after epoch 14: Top8: 80.97%
[47.1004942339374, 59.73640856672158, 66.9851729818781, 71.86161449752883, 74.69522240527183, 77.2487644151565, 79.24217462932455, 80.97199341021417]
Accuracy after epoch 

Epoch 15:   0%|          | 0/8599 [00:00<?, ?it/s]

Validating:   0%|          | 0/190 [00:00<?, ?it/s]

Accuracy after epoch 15: Top1: 46.51%
[46.50741350906095]
Accuracy after epoch 15: Top2: 58.96%
[46.50741350906095, 58.96210873146623]
Accuracy after epoch 15: Top3: 66.71%
[46.50741350906095, 58.96210873146623, 66.70510708401977]
Accuracy after epoch 15: Top4: 70.87%
[46.50741350906095, 58.96210873146623, 66.70510708401977, 70.87314662273477]
Accuracy after epoch 15: Top5: 73.92%
[46.50741350906095, 58.96210873146623, 66.70510708401977, 70.87314662273477, 73.92092257001647]
Accuracy after epoch 15: Top6: 76.19%
[46.50741350906095, 58.96210873146623, 66.70510708401977, 70.87314662273477, 73.92092257001647, 76.19439868204283]
Accuracy after epoch 15: Top7: 78.15%
[46.50741350906095, 58.96210873146623, 66.70510708401977, 70.87314662273477, 73.92092257001647, 76.19439868204283, 78.15485996705107]
Accuracy after epoch 15: Top8: 79.80%
[46.50741350906095, 58.96210873146623, 66.70510708401977, 70.87314662273477, 73.92092257001647, 76.19439868204283, 78.15485996705107, 79.80230642504118]
Accu

然后我们再来一遍同样一模一样的操作

In [11]:
import os
import torch
from tqdm.auto import tqdm
import warnings
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd

# 忽略特定的警告
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned*")

# 加载新的数据集
new_file_path = './dataset/Eclipse_total_processed_jump_final_deleted_lowdeveloper_again.csv'
# 指定需要提取的列
columns_to_extract = ['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority', 'developer',  'status']
# columns_to_extract = [ 'description', 'developer']
df = pd.read_csv(new_file_path, usecols=columns_to_extract, encoding='latin-1')

# 将developer列作为标签
label_dict = {label: idx for idx, label in enumerate(df['developer'].unique())}
df['label'] = df['developer'].replace(label_dict)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19256\2783728001.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['developer'].replace(label_dict)


In [12]:
# 合并文本信息为模型的输入，除了developer列
# df['text_input'] = df[['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority',  'status']].astype(str).agg(' '.join, axis=1)
df['text_input'] =   df['abstracts'].astype(str)+ " " + df['description'].astype(str)  # 使用空格作为分隔符

# df['text_input'] = df[[ 'product',   'component', 'severity', 'priority',  'status','description']].astype(str).agg(' '.join, axis=1)
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15, random_state=42, stratify=df.label.values)
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [13]:

# 对训练和验证数据的合并文本进行编码
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_input.values,  # 使用合并后的文本
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_input.values,  # 使用合并后的文本
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

# 准备Tensor数据
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# 定义DataLoader
batch_size = 4
train_loader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
val_loader = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=32)

In [14]:
# # 初始化模型
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)
# optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# # 加载模型
# # 计算层数
# num_transformer_layers = len(model.bert.encoder.layer)
# print(f'The BERT model has {num_transformer_layers} transformer layers.')
# print(model)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# 加载模型
# 计算层数
num_transformer_layers = len(model.bert.encoder.layer)
print(f'The BERT model has {num_transformer_layers} transformer layers.')
print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The BERT model has 12 transformer layers.
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

c:\ProgramData\anaconda3\envs\django5\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

checkpoint_path = 'model_checkpoint__eclipseprocessed_again_withabstract+description_jump.pth'

# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')



Starting training from scratch


In [16]:
from datetime import datetime

# 数据库连接信息
host = '38.147.173.234'
user = 'root'
password = '123456'
db = 'training_statistics_db'

# 模型名称，根据实际情况手动设置
model_name = 'bert-base-cased'
# 学习率和可选特性，根据实际情况手动设置
learning_rate = 1e-5  # 示例学习率
optional_feature = 'abstract+descrition(hasbeenprocessedbyjump)'  # 示例可选特性
dataset = new_file_path
num_epochs=15

for epoch in range(start_epoch, num_epochs):
    model.train()
    start_time = datetime.now()
    # ...训练过程...
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
        
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
    model.eval()
    correct_topk = {k: 0 for k in range(1, 11)}
    total = 0
    val_progress_bar = tqdm(val_loader, desc="Validating")
    
    for batch in val_progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        total += labels.size(0)
        
        # 计算top1到top10的正确率
        _, predicted_topk = torch.topk(logits, k=10, dim=1)
        labels_expanded = labels.unsqueeze(1)
        for k in range(1, 11):
            correct_topk[k] += (predicted_topk[:, :k] == labels_expanded).any(dim=1).sum().item()
                
    # 打印每个topK的准确率
    top10accuracy = []  # 初始化存储Top1到Top10准确率的数组

    for k in range(1, 11):
        accuracy = 100 * correct_topk[k] / total
        top10accuracy.append(accuracy)  # 将计算出的准确率添加到数组中
        print(f'Accuracy after epoch {epoch + 1}: Top{k}: {accuracy:.2f}%')
        print(top10accuracy)
    import pandas as pd
    import os

    # 定义数据字典，用于创建DataFrame
    data = {
            'epoch': [epoch],
            'start_time': [start_time],
            'end_time': [end_time],
            'duration': [duration],
            'user_id': [1],
            'model': [model_name],
            'top1_accuracy': [top10accuracy[0]],
            'top2_accuracy': [top10accuracy[1]],
            'top3_accuracy': [top10accuracy[2]],
            'top4_accuracy': [top10accuracy[3]],
            'top5_accuracy': [top10accuracy[4]],
            'top6_accuracy': [top10accuracy[5]],
            'top7_accuracy': [top10accuracy[6]],
            'top8_accuracy': [top10accuracy[7]],
            'top9_accuracy': [top10accuracy[8]],
            'top10_accuracy': [top10accuracy[9]],
            'optional_feature': [optional_feature],
            'learning_rate': [learning_rate],
            'dataset': [dataset]
    }

        # 创建DataFrame
    df = pd.DataFrame(data)

        # 检查train.csv文件是否存在来决定是否添加表头
    file_exists = os.path.isfile('train.csv')

        # 如果文件存在，不写入表头，模式为追加；如果文件不存在，写入表头，模式为写入
    df.to_csv('train.csv', mode='a', header=not file_exists, index=False)

    print(f'Epoch {epoch + 1} training data inserted into train.csv.')

Epoch 1:   0%|          | 0/3536 [00:00<?, ?it/s]

Validating:   0%|          | 0/78 [00:00<?, ?it/s]

Accuracy after epoch 1: Top1: 21.43%
[21.434294871794872]
Accuracy after epoch 1: Top2: 30.93%
[21.434294871794872, 30.92948717948718]
Accuracy after epoch 1: Top3: 37.66%
[21.434294871794872, 30.92948717948718, 37.66025641025641]
Accuracy after epoch 1: Top4: 41.39%
[21.434294871794872, 30.92948717948718, 37.66025641025641, 41.38621794871795]
Accuracy after epoch 1: Top5: 44.79%
[21.434294871794872, 30.92948717948718, 37.66025641025641, 41.38621794871795, 44.791666666666664]
Accuracy after epoch 1: Top6: 47.64%
[21.434294871794872, 30.92948717948718, 37.66025641025641, 41.38621794871795, 44.791666666666664, 47.63621794871795]
Accuracy after epoch 1: Top7: 50.84%
[21.434294871794872, 30.92948717948718, 37.66025641025641, 41.38621794871795, 44.791666666666664, 47.63621794871795, 50.84134615384615]
Accuracy after epoch 1: Top8: 54.13%
[21.434294871794872, 30.92948717948718, 37.66025641025641, 41.38621794871795, 44.791666666666664, 47.63621794871795, 50.84134615384615, 54.12660256410256]


Epoch 2:   0%|          | 0/3536 [00:00<?, ?it/s]

Validating:   0%|          | 0/78 [00:00<?, ?it/s]

...

Epoch 8:   0%|          | 0/3536 [00:00<?, ?it/s]

Validating:   0%|          | 0/78 [00:00<?, ?it/s]

Accuracy after epoch 8: Top1: 39.86%
[39.86378205128205]
Accuracy after epoch 8: Top2: 54.85%
[39.86378205128205, 54.84775641025641]
Accuracy after epoch 8: Top3: 64.14%
[39.86378205128205, 54.84775641025641, 64.1426282051282]
Accuracy after epoch 8: Top4: 69.35%
[39.86378205128205, 54.84775641025641, 64.1426282051282, 69.35096153846153]
Accuracy after epoch 8: Top5: 73.48%
[39.86378205128205, 54.84775641025641, 64.1426282051282, 69.35096153846153, 73.4775641025641]
Accuracy after epoch 8: Top6: 76.48%
[39.86378205128205, 54.84775641025641, 64.1426282051282, 69.35096153846153, 73.4775641025641, 76.4823717948718]
Accuracy after epoch 8: Top7: 78.41%
[39.86378205128205, 54.84775641025641, 64.1426282051282, 69.35096153846153, 73.4775641025641, 76.4823717948718, 78.40544871794872]
Accuracy after epoch 8: Top8: 79.97%
[39.86378205128205, 54.84775641025641, 64.1426282051282, 69.35096153846153, 73.4775641025641, 76.4823717948718, 78.40544871794872, 79.96794871794872]
Accuracy after epoch 8: T

Epoch 9:   0%|          | 0/3536 [00:00<?, ?it/s]

KeyboardInterrupt: 